In [3]:
import os

In [4]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [5]:
load_dotenv()

True

In [7]:
mykey=os.getenv("OPENAI_API_KEY")

In [9]:
llm=ChatOpenAI(openai_api_key=mykey,model_name="gpt-3.5-turbo",temperature=0.7)

In [16]:
with open("C:/Users/anshu/OneDrive/Desktop/chaku/GenAI/GenAI/Response.json","r") as f:
    RESPONSE_JSON=json.load(f)

In [17]:
print(RESPONSE_JSON)

{'quiz': {'text': '{text}', 'subject': '{subject}', 'tone': '{tone}', 'questions': [{'question': 'Question 1 text based on the provided content', 'options': ['Option 1', 'Option 2', 'Option 3', 'Option 4'], 'correct_answer': 'Option 1'}, {'question': 'Question 2 text based on the provided content', 'options': ['Option 1', 'Option 2', 'Option 3', 'Option 4'], 'correct_answer': 'Option 2'}]}}


In [18]:
TEMPLATE="""
TEXT:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone}.\
Make sure the questions are not repeated and check all the questions to conforming.\
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}
"""

In [21]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=TEMPLATE
)

In [22]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [24]:
TEMPLATE2="""
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {subject} multiple choice questions to check for correct English usage. \
Make sure the questions are not repeated and check all the questions to ensure they conform to the standards. \
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make quiz MCQs:
{quiz}
"""

In [25]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [26]:
review_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)